In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import batman
import emcee
import corner
from scipy.optimize import minimize
from astropy.timeseries import BoxLeastSquares

# ==========================================
# 1. GENERATE SYNTHETIC DATA
# ==========================================
np.random.seed(42)

# True parameters
true_params = {
    "rp": 0.1,       # Radius ratio
    "a": 15.0,       # Semi-major axis (a/R*)
    "inc": 88.5,     # Inclination
    "t0": 0.0,       # Mid-transit time
    "per": 1.0,      # Period
    "ecc": 0.1,      # Eccentricity
    "w": 90.0,       # Argument of periastron
    "u": [0.3, 0.2], # Limb darkening (fixed for this test)
    "sigma": 3e-4    # Noise level
}

# Generate time and flux
t = np.linspace(-0.2, 2.2, 10000)

def get_model(theta, time):
    rp, a, inc, t0, per, ecs, ess, _ = theta
    
    # Convert fitting params (ecs, ess) back to physical (ecc, w)
    ecc = ecs**2 + ess**2
    w_deg = np.degrees(np.arctan2(ess, ecs))
    if w_deg < 0: w_deg += 360.0

    # Sanity check for the batman model
    if ecc >= 1.0 or a <= 0: return np.ones_like(time)

    params = batman.TransitParams()
    params.t0 = t0
    params.per = per
    params.rp = rp
    params.a = a
    params.inc = inc
    params.ecc = ecc
    params.w = w_deg
    params.u = [0.3, 0.2]
    params.limb_dark = "quadratic"
    
    m = batman.TransitModel(params, time)
    return m.light_curve(params)

# Create True Flux
true_ecs = np.sqrt(true_params["ecc"]) * np.cos(np.radians(true_params["w"]))
true_ess = np.sqrt(true_params["ecc"]) * np.sin(np.radians(true_params["w"]))
theta_true = [true_params["rp"], true_params["a"], true_params["inc"], true_params["t0"], 
              true_params["per"], true_ecs, true_ess, np.log(true_params["sigma"])]

flux_true = get_model(theta_true, t)
flux = flux_true + np.random.normal(0, true_params["sigma"], size=len(t))
flux_err = np.ones_like(t) * 1e-4 # Assume telescope precision is 100 ppm

# ==========================================
# 2. BLS SEARCH (Find P and T0)
# ==========================================
print("--- 1. BLS SEARCH ---")
model_bls = BoxLeastSquares(t, flux)
period_grid = np.linspace(0.5, 2.0, 5000)
periodogram = model_bls.power(period_grid, duration=0.1, objective="snr")

best_ix = np.argmax(periodogram.power)
bls_per = periodogram.period[best_ix]
bls_t0 = periodogram.transit_time[best_ix]
bls_depth = periodogram.depth[best_ix]
bls_rp = np.sqrt(bls_depth)

print(f"BLS Found: P={bls_per:.4f}, T0={bls_t0:.4f}, Rp~{bls_rp:.4f}")

# ==========================================
# 3. STRESS TEST: SCRAMBLE THE GUESSES
# ==========================================
print("\n--- 2. STRESS TESTING (SCRAMBLING GUESSES) ---")

# We intentionally offset the parameters to see if the optimizer can fix it.
# A human guess for 'a' is often derived from Kepler's law assuming circular orbit.
rho_estimate = (bls_per)**(-2) # Very rough density proxy
guess_a = 10.0 # Intentionally wrong (True is 15.0)
guess_inc = 85.0 # Intentionally wrong (True is 88.5)

# Scramble Period and T0 slightly (simulate limited resolution of BLS)
guess_per = bls_per + 0.001 # Offset by ~1.5 minutes
guess_t0 = bls_t0 - 0.005   # Offset by ~7 minutes

# Guess Circular orbit (standard starting point)
guess_ecs = 0.0
guess_ess = 0.0

initial_guess = np.array([
    bls_rp,    # Rp
    guess_a,   # a
    guess_inc, # inc
    guess_t0,  # t0 (Perturbed)
    guess_per, # per (Perturbed)
    guess_ecs, # ecs
    guess_ess, # ess
    np.log(1e-3) # ln_sigma
])

print(f"Bad Guess P  : {guess_per:.5f} (True: {true_params['per']})")
print(f"Bad Guess T0 : {guess_t0:.5f} (True: {true_params['t0']})")
print(f"Bad Guess a  : {guess_a:.1f}   (True: {true_params['a']})")

# ==========================================
# 4. OPTIMIZATION (THE FIX)
# ==========================================
print("\n--- 3. RUNNING OPTIMIZER ---")

rho_obs_mean = (true_params["a"]**3) / (true_params["per"]**2)
rho_obs_sigma = 0.1 * rho_obs_mean  # Allow 10% uncertainty in stellar density

def ln_prior(theta):
    rp, a, inc, t0, per, ecs, ess, ln_sigma = theta
    
    # --- Standard Hard Bounds ---
    if not (0.01 < rp < 1.0): return -np.inf
    if not (1.0 < a < 100.0): return -np.inf
    if not (60.0 < inc < 90.0): return -np.inf
    if not (t.min() < t0 < t.max()): return -np.inf
    if not (0.2 < per < 3.0): return -np.inf
    if not (-15.0 < ln_sigma < 1.0): return -np.inf
    
    # --- Eccentricity Check ---
    ecc_sq = ecs**2 + ess**2
    if ecc_sq >= 0.9: return -np.inf
    
    # --- THE FIX: Stellar Density Constraint ---
    # This prevents 'a' from drifting to 20 unless the period changes drastically
    # or the eccentricity matches the density perfectly (breaking the degeneracy).
    rho_model = (a**3) / (per**2)
    lp_density = -0.5 * ((rho_model - rho_obs_mean) / rho_obs_sigma)**2
    
    return lp_density

def ln_likelihood_gaussian(theta, time, flux, flux_err):
    """
    Gaussian Likelihood that incorporates both instrumental error (flux_err)
    and an unknown noise term (jitter).
    """
    rp, a, inc, t0, per, ecs, ess, ln_sigma_jitter = theta
    
    # 1. Unpack the Jitter (this is the 'sigma' we are fitting for)
    sigma_jitter = np.exp(ln_sigma_jitter)
    
    # 2. Combine Jitter with Known Data Errors (added in quadrature)
    #    This creates the total noise budget for each data point.
    sigma_total_sq = flux_err**2 + sigma_jitter**2
    
    # 3. Calculate Model (with safety checks)
    ecc_sq = ecs**2 + ess**2
    if ecc_sq >= 0.9: return -np.inf

    w_deg = np.degrees(np.arctan2(ess, ecs))
    if w_deg < 0: w_deg += 360.0

    try:
        model = batman_model(time, rp, a, inc, t0, per, ecc_sq, w_deg)
    except:
        return -np.inf

    # 4. Calculate Residuals
    resid = flux - model

    # 5. THE GAUSSIAN LOG-LIKELIHOOD
    #    Term A: Chi-squared weighted by total variance
    #    Term B: Penalty term for the noise model (prevents sigma from going to infinity)
    ln_like = -0.5 * np.sum( (resid**2 / sigma_total_sq) + np.log(2.0 * np.pi * sigma_total_sq) )
    
    return ln_like

def neg_ln_posterior(theta, time, flux, flux_err):
    lp = ln_prior(theta)
    if not np.isfinite(lp): return 1e25
    ll = ln_likelihood_gaussian(theta, time, flux, flux_err)
    return -(lp + ll)

# Run Minimization
soln = minimize(neg_ln_posterior, initial_guess, 
                args=(t, flux, flux_err), # <--- ADDED flux_err
                method='Nelder-Mead')

print(f"Optimization Success: {soln.success}")
print(f"Recovered P  : {soln.x[4]:.5f}")
print(f"Recovered T0 : {soln.x[3]:.5f}")
print(f"Recovered a  : {soln.x[1]:.5f}")

# Visualization of Correction
plt.figure(figsize=(10, 5))
plt.plot(t, flux, "k.", alpha=0.1, label="Data")
# Plot the BAD guess
bad_model = get_model(initial_guess, t)
plt.plot(t, bad_model, "r--", lw=1, alpha=0.5, label="Initial (Bad) Guess")
# Plot the OPTIMIZED guess
opt_model = get_model(soln.x, t)
plt.plot(t, opt_model, "b-", lw=2, label="Optimized Fit")
plt.xlim(-0.1, 0.1) # Zoom in on first transit
plt.title("Visualizing the Optimization Step (Zoomed)")
plt.legend()
plt.show()

# ==========================================
# 5. MCMC
# ==========================================
print("\n--- 4. RUNNING MCMC ---")

def ln_posterior(theta, time, flux, flux_err):
    lp = ln_prior(theta)
    if not np.isfinite(lp): return -np.inf
    ll = ln_likelihood_gaussian(theta, time, flux, flux_err)
    return lp + ll

ndim = 8
nwalkers = 50

# Initialize in a tiny ball around the OPTIMIZED solution
pos = soln.x + 1e-4 * np.random.randn(nwalkers, ndim)

sampler = emcee.EnsembleSampler(nwalkers, ndim, ln_posterior, 
                                args=(t, flux, flux_err)) # <--- ADDED flux_err
sampler.run_mcmc(pos, 3000, progress=True)

# ==========================================
# 6. RESULTS
# ==========================================
flat_samples = sampler.get_chain(discard=1000, thin=15, flat=True)

# Process samples for Corner Plot
# Convert fitting params to physical params
rp = flat_samples[:, 0]
a = flat_samples[:, 1]
inc = flat_samples[:, 2]
t0 = flat_samples[:, 3]
per = flat_samples[:, 4]
ecc = flat_samples[:, 5]**2 + flat_samples[:, 6]**2
w = np.degrees(np.arctan2(flat_samples[:, 6], flat_samples[:, 5]))
w[w<0] += 360
sigma = np.exp(flat_samples[:, 7])

samples_phys = np.vstack([rp, a, inc, t0, per, ecc, w, sigma]).T
labels = ["rp", "a", "inc", "t0", "per", "ecc", "w", "sigma"]
truths = [true_params["rp"], true_params["a"], true_params["inc"], true_params["t0"], 
          true_params["per"], true_params["ecc"], true_params["w"], true_params["sigma"]]


corner.corner(samples_phys, labels=labels, truths=truths, show_titles=True, title_fmt=".4f")
plt.show()